Project: /_project.yaml
Book: /_book.yaml
Keywords: examples,gemini,beginner,googleai,quickstart,python,text,chat,vision,embed
Image_path: /static/site-assets/images/docs/logo-python.svg

<link rel="stylesheet" href="/site-assets/css/style.css">

# Gemini API: Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://developers.generativeai.google/tutorials/text_quickstart"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This quickstart demonstrates how to use the Python SDK for the Gemini API, which gives you access to Google's Gemini large language models. In this quickstart, you will learn how to:

1. Set up your development environment and API access to use Gemini.
2. Generate text responses from text inputs.
3. Generate text responses from multimodal inputs (text and images).
4. Use Gemini for multi-turn conversations (chat).
5. Use embeddings for large language models.

## Prerequisites

You can run this quickstart in Google Colab.

To complete this quickstart on your own development environment, ensure that your envirmonement meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.


## Setup



### Install the Python SDK

The Python SDK for the Gamini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:


In [ ]:
!pip install -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.1
    Uninstalling google-generativeai-0.3.1:
      Successfully uninstalled google-generativeai-0.3.1


### Import packages

Import the necessary packages.

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in MakerSuite.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`



In [ ]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('API_KEY')

genai.configure(api_key=API_KEY)

## Function calls

In [ ]:
import google.ai.generativelanguage as glm

In [ ]:
model = genai.GenerativeModel(
    'gemini-pro',
    tools=[
        glm.Tool(
            function_declarations=[
              glm.FunctionDeclaration(
                name='datetime',
                description="Returns the current UTC date and time."
              )
            ]
        )
    ])
chat = model.start_chat()

response = chat.send_message(
    'How many days until Christmas',
)

In [ ]:
response.candidates

[content {
  parts {
    function_call {
      name: "datetime"
      args {
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
]

In [ ]:
response = chat.send_message(
  glm.Content(
    parts=[glm.Part(
        function_response = glm.FunctionResponse(
          name='datetime',
          response={'datetime': 'Sun Dec 5 03:33:56 PM UTC 2023'}
        )
    )]
  )
)

In [ ]:
response.text

' Christmas is on Sunday Dec 25, 2023. So, there are 20 days until Christmas.'

In [ ]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat()

response = chat.send_message(
    "What's 2312371 * 234234 ?",

)

In [ ]:
2312371 * 234234

541635908814

In [ ]:
response.text

'544156360802'

In [ ]:
2312371 * 234234 - int(response.text)

-2520451988

In [ ]:
calculator = glm.Tool(
    function_declarations=[
      glm.FunctionDeclaration(
        name='Add',
        description="Returns sum of two numbers.",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'a': glm.Schema(type=glm.Type.NUMBER),
                'b': glm.Schema(type=glm.Type.NUMBER)
            }
        )
      ),
      glm.FunctionDeclaration(
        name='Multiply',
        description="Returns product of two numbers.",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'a':glm.Schema(type=glm.Type.NUMBER),
                'b':glm.Schema(type=glm.Type.NUMBER)
            }

        )
      )
    ])

In [ ]:
model = genai.GenerativeModel('gemini-pro', tools=[calculator])
chat = model.start_chat()

response = chat.send_message(
    "What's 2312371 * 234234 ?",

)

In [ ]:

response.candidates

[content {
  parts {
    function_call {
      name: "Multiply"
      args {
        fields {
          key: "a"
          value {
            number_value: 2312371.0
          }
        }
        fields {
          key: "b"
          value {
            number_value: 234234.0
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
]

## Grounded question answering

In [ ]:
import pathlib
from google.colab import userdata

oauth = userdata.get('PALM_OAUTH')
pathlib.Path('client_secret.json').write_text(oauth);

In [ ]:
!gcloud auth application-default login --no-browser --client-id-file=client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.retriever'


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=847260786417-t6tgju7vmp1ojuc71m5nkmhr1p4o342e.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-langua

In [ ]:
import google.ai.generativelanguage as glm

In [ ]:
client = glm.GenerativeServiceClient()

In [ ]:
empty_request = glm.GenerateAnswerRequest()
[name for name in dir(empty_request) if not name.startswith('__')]

['AnswerStyle',
 'answer_style',
 'contents',
 'inline_passages',
 'model',
 'safety_settings',
 'semantic_retriever',
 'temperature']

In [ ]:
for m in genai.list_models():
  if 'generateAnswer' in m.supported_generation_methods:
    print(m)

Model(name='models/aqa',
      base_model_id='',
      version='001',
      display_name='Model that performs Attributed Question Answering.',
      description=('Model trained to return answers to questions that are grounded in provided '
                   'sources, along with estimating answerable probability.'),
      input_token_limit=7168,
      output_token_limit=1024,
      supported_generation_methods=['generateAnswer'],
      temperature=0.2,
      top_p=1.0,
      top_k=40)


In [ ]:
client.generate_answer(
    glm.GenerateAnswerRequest(
        model='models/aqa',
        contents=[
            {'parts':[{'text':"What time is it?"}]}
        ],
        inline_passages=glm.GroundingPassages(
            passages=[
                glm.GroundingPassage(id='1', content={'parts':[{'text': "It's Thursday"}]}),
                glm.GroundingPassage(id='2', content={'parts':[{'text': "It's December"}]}),
                glm.GroundingPassage(id='3', content={'parts':[{'text': "It's lunch time"}]}),
                glm.GroundingPassage(id='4', content={'parts':[{'text': "It's raining"}]}),
            ]
        )
    )
)

NotFound: ignored



## What's next

-   Prompt design is the process of creating prompts that elicit the desired response from language models. Writing well structured prompts is an essential part of ensuring accurate, high quality responses from a language model. Learn about best practices for [prompt writing](/docs/prompt_best_practices.md).
-   Gemini offers several model variations to meet the needs of different use cases, such as input types and complexity, implementations for chat or other dialog language tasks, and size constraints. Learn about the available [Gemini models](/models/language.md).